# Enorm Dienstplan automatisiert erstellen

## 0. Import libraries and data

In [1]:
import numpy as np
import pandas as pd


In [2]:
#Shift possibilities

#shifts
#'Früh':'08-13' III
#'Früh':'08-14' XV -> check
#'Früh':'08-15'
#'Früh':'08-16' XII -> check
#'Früh':'08-18' II
#'Früh':'08-19'
#'Früh':'09-15' I
#'Früh':'09-14' II
#'Früh':'09-16'
#'Früh':'09-17' IIII -> check
#'Früh':'09-18' I
#'Früh':'09-19' XII -> check
#'Früh':'10-14'
#'Früh':'10-18' VI -> check
#'Früh':'10-19' III
#'Früh':'10-20' VIIII
#'Früh':'10-21' IIII
#'Früh':'11-20' I
#'Früh':'12-21' XXII -> check
#'Früh':'12-20'
#'Spät':'10-13'
#'Spät':'13-23' II
#'Spät':'14-21' II
#'Spät':'15-23' XXIIII -> check
#'Spät':'17-23' III
#'Nacht':'14-10' IIII
#'Nacht':'14-11' V
#'Nacht':'17-10'
#'Nacht':'17-11'
#'Nacht':'17-13'
#'Nacht':'19-10' VI
#'Nacht':'13-13' VII


### Import employee data

In [3]:
#As excel with columns
mitarbeitende_df = pd.read_excel('data/Input Mitarbeitende_spalten.xlsx')

## 1. Create simple Dienstplan

Create a dataframe with 8 hour slots ('Schichten') in a 24 hour day for a 30 days month where from the dataframe wochenstunden_df each employee ('Mitarbeitenden_ID') is randomly spread into the 'Schichten' and they have at least 11 hours breaks between their 'Schichten' and they are close to working their number of working hours per week ('Wochenstunden') found in wochenstunden_df. 

### Assign based on regulations in Dienst 1 and Dienst 2

In [27]:
import pandas as pd
from ortools.sat.python import cp_model

# Load data from both sheets
mitarbeitende_df = pd.read_excel('data/Input Mitarbeitende_spalten.xlsx')
mitarbeitende_info_df = pd.read_excel('data/Input Mitarbeitende_spalten.xlsx', sheet_name='MA_Info')

# Filter for workers at 'Enorm'
enorm_workers = mitarbeitende_info_df[mitarbeitende_info_df['Dienstbedingungen'] == 'Enorm']

# Merge to get availability information
merged_df = pd.merge(enorm_workers, mitarbeitende_df, on='Mitarbeitenden_ID')

# Shift types for weekdays and weekends
shift_types = {
    'Früh 1': '08-16',
    'Früh 2': '08-14',
    'Früh 3': '10-18',
    'Büro': '09-18',
    'Früh 5': '12-21',
    'Spät': '15-23',
    'Nacht': '19-10',
}

shift_types_weekend = {
    'Früh 1': '09-17',
    'Früh 2': '09-19',
    'Früh 3': '12-21',
    'Spät': '15-23',
    'Nacht': '19-11'
}

# Parameters
month = 7
year = 2024

# Create the number of days per month
if month in [1, 3, 5, 7, 8, 10, 12]:
    num_days = 31
elif month in [4, 6, 9, 11]:
    num_days = 30
elif month == 2:
    num_days = 29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28
else:
    raise ValueError("Invalid month")

# Generate the date range
dates = pd.date_range(start=f'{year}-{month:02d}-01', periods=num_days)

# Declare parameters
workers = len(enorm_workers)
days = num_days
shifts_weekday = len(shift_types)
shifts_weekend = len(shift_types_weekend)
min_break = 11  # Minimum break between shifts in hours

# Initialize the CP model
model = cp_model.CpModel()

# Create shift variables
shift_options = {}
for day in range(days):
    num_shifts = shifts_weekday if dates[day].weekday() < 5 else shifts_weekend
    for shift in range(num_shifts):
        for worker in range(workers):
            shift_options[(day, shift, worker)] = model.NewBoolVar(f'shift_{day}_{shift}_{worker}')

# Constraint: Each shift is assigned to exactly one worker
for day in range(days):
    num_shifts = shifts_weekday if dates[day].weekday() < 5 else shifts_weekend
    for shift in range(num_shifts):
        model.Add(sum(shift_options[(day, shift, worker)] for worker in range(workers)) == 1)

# Constraint: Each worker works at most one shift per day
for day in range(days):
    for worker in range(workers):
        num_shifts = shifts_weekday if dates[day].weekday() < 5 else shifts_weekend
        model.Add(sum(shift_options[(day, shift, worker)] for shift in range(num_shifts)) <= 1)

# Helper function to get shift end time
def get_shift_end_time(shift_name, is_weekend):
    shift_times = shift_types_weekend if is_weekend else shift_types
    shift_time = shift_times[shift_name]
    start_hour, end_hour = map(int, shift_time.split('-'))
    if end_hour < start_hour:  # Shift ends the next day
        end_hour += 24
    return end_hour

# Custom break constraints
for worker in range(workers):
    for day in range(days):
        num_shifts = shifts_weekday if dates[day].weekday() < 5 else shifts_weekend
        for shift in range(num_shifts):
            shift_name = list(shift_types.keys())[shift] if dates[day].weekday() < 5 else list(shift_types_weekend.keys())[shift]
            shift_end_hour = get_shift_end_time(shift_name, dates[day].weekday() >= 5)
            for next_day in range(day + 1, days):
                next_num_shifts = shifts_weekday if dates[next_day].weekday() < 5 else shifts_weekend
                for next_shift in range(next_num_shifts):
                    next_shift_name = list(shift_types.keys())[next_shift] if dates[next_day].weekday() < 5 else list(shift_types_weekend.keys())[next_shift]
                    next_shift_start_hour = int((shift_types if dates[next_day].weekday() < 5 else shift_types_weekend)[next_shift_name].split('-')[0])
                    if (next_day - day) * 24 + next_shift_start_hour - shift_end_hour < min_break:
                        model.Add(shift_options[(day, shift, worker)] + shift_options[(next_day, next_shift, worker)] <= 1)

# Constraint: Handle 'WF' (wish to not work) and 'U' (vacation day) constraints
for worker_index, row in merged_df.iterrows():
    for day in range(days):
        if row.iloc[day + 5] in ['WF', 'U']:  # +5 to skip the initial columns
            num_shifts = shifts_weekday if dates[day].weekday() < 5 else shifts_weekend
            for shift in range(num_shifts):
                model.Add(shift_options[(day, shift, worker_index)] == 0)

# Helper function to calculate hours from shift time
def calculate_shift_hours(shift_time):
    start_hour, end_hour = map(int, shift_time.split('-'))
    if end_hour < start_hour:
        end_hour += 24
    return end_hour - start_hour

# Track actual hours worked per worker per month
total_hours = {}
for worker in range(workers):
    total_hours[worker] = model.NewIntVar(0, merged_df.at[worker, 'Wochenstunden'] + 20, f'total_hours_worker_{worker}')

    # Calculate actual hours worked based on assigned shifts
    for day in range(days):
        num_shifts = shifts_weekday if dates[day].weekday() < 5 else shifts_weekend
        for shift in range(num_shifts):
            shift_name = list(shift_types.keys())[shift] if dates[day].weekday() < 5 else list(shift_types_weekend.keys())[shift]
            shift_time = shift_types[shift_name] if dates[day].weekday() < 5 else shift_types_weekend[shift_name]
            hours_worked = calculate_shift_hours(shift_time)
            total_hours[worker] += shift_options[(day, shift, worker)] * hours_worked

# Constraint: Limit total hours worked per month to 'Wochenstunden' +/- 20 hours
for worker in range(workers):
    target_hours_per_week = merged_df.at[worker, 'Wochenstunden']
    target_hours = target_hours_per_week / 7 * num_days  # Total target hours for the month

    # Convert target_hours to an integer if necessary
    target_hours_int = int(round(target_hours)) if isinstance(target_hours, float) else target_hours

    # Create a variable for the actual total hours worked
    total_hours_worked = total_hours[worker]

    # Ensure the total hours constraint
    model.Add(total_hours_worked >= target_hours_int - 20)
    model.Add(total_hours_worked <= target_hours_int + 20)


class SolutionPrinterClass(cp_model.CpSolverSolutionCallback):
    def __init__(self, shift_options, workers, days, shifts_weekday, shifts_weekend, sols, dates):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shift_options = shift_options
        self._workers = workers
        self._days = days
        self._shifts_weekday = shifts_weekday
        self._shifts_weekend = shifts_weekend
        self._solutions = set(sols)
        self._solution_count = 0
        self._dates = dates
        self._results = []

    def on_solution_callback(self):
        if self._solution_count in self._solutions:
            for day in range(self._days):
                num_shifts = self._shifts_weekday if self._dates[day].weekday() < 5 else self._shifts_weekend
                for shift in range(num_shifts):
                    for worker in range(self._workers):
                        if self.Value(self._shift_options[(day, shift, worker)]):
                            shift_name = list(shift_types.keys())[shift] if self._dates[day].weekday() < 5 else list(shift_types_weekend.keys())[shift]
                            shift_time = shift_types[shift_name] if self._dates[day].weekday() < 5 else shift_types_weekend[shift_name]
                            worker_id = merged_df.iloc[worker]['Mitarbeitenden_ID']
                            self._results.append((self._dates[day], shift_name, shift_time, worker_id))
        self._solution_count += 1

    def get_dataframe(self):
        return pd.DataFrame(self._results, columns=['Datum', 'Schicht', 'Schichtzeit', 'Dienst'])

# Solve the model
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solver.parameters.log_search_progress = True  # Log progress
solution_printer = SolutionPrinterClass(shift_options, workers, days, shifts_weekday, shifts_weekend, range(1), dates)

status = solver.Solve(model, solution_printer)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    # Get and display the results as a DataFrame
    df = solution_printer.get_dataframe()
    print(df)
else:
    print("No solution found.")



Starting CP-SAT solver v9.4.1874
Parameters: log_search_progress: true linearization_level: 0
Setting number of workers to 8

Initial satisfaction model '':
#Variables: 4444
  - 4422 Booleans in [0,1]
  - 1 in [0,24]
  - 1 in [0,25]
  - 3 in [0,30]
  - 5 in [0,40]
  - 5 in [0,50]
  - 7 in [0,60]
#kLinear1: 442
#kLinear2: 6072
#kLinearN: 927 (#terms: 17732)

Starting presolve at 0.01s
INFEASIBLE: ''
Unsat after presolving constraint #7425 (warning, dump might be inconsistent): linear { vars: 4436 coeffs: 1 domain: 69 domain: 9223372036854775807 }

Presolve summary:
  - 0 affine relations were detected.
  - rule 'TODO linear2: contains a Boolean.' was applied 10838 times.
  - rule 'bool_or: implications' was applied 5419 times.
  - rule 'linear: always true' was applied 721 times.
  - rule 'linear: empty' was applied 442 times.
  - rule 'linear: fixed or dup variables' was applied 1371 times.
  - rule 'linear: infeasible' was applied 1 time.
  - rule 'linear: negative clause' was applie

In [21]:
print(df.head(50))

        Datum Schicht Schichtzeit  Dienst
0  2024-07-01  Früh 1       08-16     127
1  2024-07-01  Früh 2       08-14     125
2  2024-07-01  Früh 3       10-18     122
3  2024-07-01    Büro       09-18     113
4  2024-07-01  Früh 5       12-21     107
5  2024-07-01    Spät       15-23     103
6  2024-07-01   Nacht       19-10     110
7  2024-07-02  Früh 1       08-16     109
8  2024-07-02  Früh 2       08-14     108
9  2024-07-02  Früh 3       10-18     106
10 2024-07-02    Büro       09-18     105
11 2024-07-02  Früh 5       12-21     104
12 2024-07-02    Spät       15-23     103
13 2024-07-02   Nacht       19-10     107
14 2024-07-03  Früh 1       08-16     110
15 2024-07-03  Früh 2       08-14     108
16 2024-07-03  Früh 3       10-18     106
17 2024-07-03    Büro       09-18     105
18 2024-07-03  Früh 5       12-21     104
19 2024-07-03    Spät       15-23     103
20 2024-07-03   Nacht       19-10     109
21 2024-07-04  Früh 1       08-16     108
22 2024-07-04  Früh 2       08-14 

In [5]:
import pandas as pd
from ortools.sat.python import cp_model

# Import data
mitarbeitende_df = pd.read_excel('data/Input Mitarbeitende_spalten.xlsx')

# Month and year
month = 7
year = 2024

# Create the number of days per month
if month in [1, 3, 5, 7, 8, 10, 12]:
    num_days = 31
elif month in [4, 6, 9, 11]:
    num_days = 30
elif month == 2:
    num_days = 29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28
else:
    raise ValueError("Invalid month")

# Generate the date range
dates = pd.date_range(start=f'{year}-{month:02d}-01', periods=num_days)

# Declare parameters
workers = len(mitarbeitende_df['Mitarbeitenden_ID'])
days = num_days
min_break = 6
maxshiftsperday = 1
maxdifference = 1

# Dictionary to map shift types to time ranges
shift_types = {
    'Früh 1': '08-16',
    'Früh 2': '08-14', # für Teilzeit
    'Früh 3': '10-18',
    'Büro': '09-18',
    'Früh 5': '12-21',
    'Spät': '15-23',
    'Nacht': '19-10',
}

shift_types_weekend = {
    'Früh 1': '09-17',
    'Früh 2': '09-19',
    'Früh 3': '12-21',
    'Spät': '15-23',
    'Nacht': '19-11'
}

# Initialize model
model = cp_model.CpModel()

# Initialize lists to store shifts for weekdays and weekends
shifts_weekday = list(shift_types.keys())  # Include all weekday shifts
shifts_weekend = list(shift_types_weekend.keys())  # Include all weekend shifts

# Iterate over each date to determine weekday or weekend
for idx, date in enumerate(dates):
    if date.weekday() >= 5:  # Weekend
        shifts = len(shifts_weekend)
        shift_type_keys = shifts_weekend
        shift_type_values = shift_types_weekend
    else:  # Weekday
        shifts = len(shifts_weekday)
        shift_type_keys = shifts_weekday
        shift_type_values = shift_types

    # Create shift options for each day
    for y in range(shifts):
        for z in range(workers):
            model.NewBoolVar(f"shift_{idx}_{y}_{z}")

# Constraint: Each shift is assigned to exactly one worker
for x in range(days):
    for y in range(shifts):
        model.Add(sum(model.NewBoolVar(f"shift_{x}_{y}_{z}") for z in range(workers)) == 1)

# Constraint: Each worker works at most one shift per day
for x in range(days):
    for z in range(workers):
        model.Add(sum(model.NewBoolVar(f"shift_{x}_{y}_{z}") for y in range(shifts)) <= maxshiftsperday)

# Custom break constraints
for z in range(workers):
    for x in range(days):
        for y in range(shifts):
            for k in range(1, min_break):
                if y + k < shifts:
                    model.Add(model.NewBoolVar(f"shift_{x}_{y}_{z}") + model.NewBoolVar(f"shift_{x}_{y + k}_{z}") <= 1)
                if x + 1 < days and y - k >= 0:
                    model.Add(model.NewBoolVar(f"shift_{x}_{y}_{z}") + model.NewBoolVar(f"shift_{x + 1}_{y - k}_{z}") <= 1)

# Constraint: Balance the number of shifts per worker
num_u_wf_days = [sum(1 for cell in mitarbeitende_df.iloc[z, 1:] if cell in ['U', 'WF']) for z in range(workers)]

total_shifts = days * shifts
shifts_per_worker = total_shifts // workers
shifts_adjustment = total_shifts % workers

for z in range(workers):
    min_shifts = shifts_per_worker
    if z < shifts_adjustment:
        min_shifts += 1

    if days - num_u_wf_days[z] >= min_shifts:
        shiftsassigned = sum(model.NewBoolVar(f"shift_{x}_{y}_{z}") for x in range(days) for y in range(shifts))
        model.Add(min_shifts <= shiftsassigned)
        model.Add(shiftsassigned <= min_shifts + maxdifference)

# Constraint: Handle 'WF' (wish to not work) and 'U' (vacation day) constraints
for z, worker_id in enumerate(mitarbeitende_df['Mitarbeitenden_ID']):
    for x in range(days):
        if mitarbeitende_df.iloc[z, x + 1] in ['WF', 'U']:
            for y in range(shifts):
                model.Add(model.NewBoolVar(f"shift_{x}_{y}_{z}") == 0)

# Custom solution printer to store results in a DataFrame
class SolutionPrinterClass(cp_model.CpSolverSolutionCallback):
    def __init__(self, model, workers, days, shifts, dates):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._model = model
        self._workers = workers
        self._days = days
        self._shifts = shifts
        self._dates = dates
        self._solutions = []
        self._solution_count = 0

    def on_solution_callback(self):
        self._solution_count += 1
        solution = []
        for x in range(self._days):
            for y in range(self._shifts):
                for z in range(self._workers):
                    if self.Value(self._model.NewBoolVar(f"shift_{x}_{y}_{z}")) == 1:
                        shift_type = shift_type_keys[y]  # Determine shift type based on weekday or weekend
                        shift_time = shift_type_values[shift_type]  # Get shift time from respective dictionary
                        solution.append((self._dates[x], shift_type, shift_time, mitarbeitende_df['Mitarbeitenden_ID'].iloc[z]))
        self._solutions.append(solution)

    def get_dataframe(self):
        if self._solution_count > 0:
            return pd.DataFrame(self._solutions[-1], columns=['Datum', 'Schicht', 'Schichtzeit', 'Dienst'])
        else:
            return pd.DataFrame(columns=['Datum', 'Schicht', 'Schichtzeit', 'Dienst'])

# Solve the model
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solution_printer = SolutionPrinterClass(model, workers, days, shifts, dates)

solver.parameters.log_search_progress = True  # Log progress

status = solver.Solve(model, solution_printer)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    # Get and display the results as a DataFrame
    df = solution_printer.get_dataframe()
    df.to_excel('Dienstplan.xlsx', index=False)
    print(df.head(20))
else:
    print("No solution found.")



Starting CP-SAT solver v9.4.1874
Parameters: log_search_progress: true linearization_level: 0
Setting number of workers to 8

Initial satisfaction model '':
#Variables: 98837
  - 98837 Booleans in [0,1]
#kLinear1: 511
#kLinear2: 36600
#kLinearN: 1203 (#terms: 25172)

Starting presolve at 0.04s
[ExtractEncodingFromLinear] #potential_supersets=0 #potential_subsets=0 #at_most_one_encodings=0 #exactly_one_encodings=0 #unique_terms=0 #multiple_terms=0 #literals=0 time=0.000785874s
[Probing] deterministic_time: 0.0152892 (limit: 1) wall_time: 0.0759346 (92278/92278)
[DetectDuplicateConstraints] #duplicates=0 #without_enforcements=0 time=0.00114866s
[DetectDominatedLinearConstraints] #relevant_constraints=0 #work_done=0 #num_inclusions=0 #num_redundant=0 time=0.000773793s
[DetectOverlappingColumns] #processed_columns=0 #work_done=0 #nz_reduction=0 time=0.00135627s
[ProcessSetPPC] #relevant_constraints=0 #num_inclusions=0 work=0 time=0.00154114s
[Symmetry] Graph for symmetry has 98837 nodes a

In [4]:
import pandas as pd
from ortools.sat.python import cp_model

#Import data
mitarbeitende_df = pd.read_excel('data/Input Mitarbeitende_spalten.xlsx')

# Month and year
month = 7
year = 2024

#Create the number of days per month
if month in [1, 3, 5, 7, 8, 10, 12]:
    num_days = 31
elif month in [4, 6, 9, 11]:
    num_days = 30
elif month == 2:
    num_days = 29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28
else:
    raise ValueError("Invalid month")

# Generate the date range
dates = pd.date_range(start=f'{year}-{month:02d}-01', periods=num_days)

# Declare parameters
workers = len(mitarbeitende_df['Mitarbeitenden_ID'])
shifts = 6
days = len(dates.strftime('%Y-%m-%d').tolist())
min_break = 6 # can't really be changed (i dont know why now)
maxshiftsperday = 1
maxdifference = 1 #difference between number of shifts per employee (needs to be changed later when hours are taken into consideration)

# Dictionary to map shift types to time ranges
shift_types = {
    'Früh 1': '08-16',
    'Früh 2': '08-14', #für Teilzeit
    'Früh 3': '10-18',
    # 'Früh 4': '09-19', #vlt wieder raus, wurde oft genommen
    'Büro': '09-18',
    'Früh 5': '12-21',
    'Spät': '15-23',
    'Nacht': '19-10',
}

shift_types_weekend = {
    'Früh 1': '09-17',
    'Früh 2': '09-19',
    'Früh 3': '12-21',
    'Spät': '15-23',
    'Nacht': '19-11'
}

# Initialize model
model = cp_model.CpModel()

# Create shift options
shiftoptions = {}
for x in range(days):
    for y in range(shifts):
        for z in range(workers):
            shiftoptions[(x, y, z)] = model.NewBoolVar(f"shift_with_id_{x}_{y}_{z}")

# Constraint: Each shift is assigned to exactly one worker
for x in range(days):
    for y in range(shifts):
        model.Add(sum(shiftoptions[(x, y, z)] for z in range(workers)) == 1)

# Constraint: Each worker works at most one shift per day
for x in range(days):
    for z in range(workers):
        model.Add(sum(shiftoptions[(x, y, z)] for y in range(shifts)) <= maxshiftsperday)


# !!!! allow for custom break based on actual hours instead of a fixed min_break
# Add constraint ensuring at least a min_break of 6 shifts between consecutive shifts for each worker
# Add custom break constraints
for z in range(workers):
    for x in range(days):
        for y in range(shifts):
            for k in range(1, min_break):  # Adjust based on the break_time
                if y + k < shifts:
                    model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x, y + k, z)] <= 1)
                if x + 1 < days and y - k >= 0:
                    model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x + 1, y - k, z)] <= 1)

# Constraint: Balance the number of shifts per worker
# Count the number of 'U' or 'WF' days for each worker
num_u_wf_days = [sum(1 for cell in mitarbeitende_df.iloc[z, 1:] if cell in ['U', 'WF']) for z in range(workers)]

# Calculate target shifts per worker based on the total shifts and number of 'U'/'WF' days
total_shifts = days * shifts
shifts_per_worker = total_shifts // workers
shifts_adjustment = total_shifts % workers

# Distribute shifts evenly among workers
for z in range(workers):
    min_shifts = shifts_per_worker
    if z < shifts_adjustment:
        min_shifts += 1

    # Add constraints only if there are enough non-'U'/'WF' days for the worker
    if days - num_u_wf_days[z] >= min_shifts:
        shiftsassigned = sum(shiftoptions[(x, y, z)] for x in range(days) for y in range(shifts))
        model.Add(min_shifts <= shiftsassigned)
        model.Add(shiftsassigned <= min_shifts + maxdifference)

# Constraint: Handle 'WF' (wish to not work) and 'U' (vacation day) constraints
for z, worker_id in enumerate(mitarbeitende_df['Mitarbeitenden_ID']):
    for x in range(days):
        if mitarbeitende_df.iloc[z, x + 1] == 'WF' or mitarbeitende_df.iloc[z, x + 1] == 'U':
            for y in range(shifts):
                model.Add(shiftoptions[(x, y, z)] == 0)

# Constraint for weekend (weekends start at 9 e.g.)

# Cnstraint for workers availability (hours and certain shift preferences (office / nights))

# Custom solution printer to store results in a DataFrame
class SolutionPrinterClass(cp_model.CpSolverSolutionCallback):
    def __init__(self, shiftoptions, workers, days, shifts, sols):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shiftoptions = shiftoptions
        self._workers = workers
        self._days = days
        self._shifts = shifts
        self._solutions = set(sols)
        self._solution_count = 0
        self._results = []

    def on_solution_callback(self):
        if self._solution_count in self._solutions:
            for x in range(self._days):
                for y in range(self._shifts):
                    for z in range(self._workers):
                        if self.Value(self._shiftoptions[(x, y, z)]):
                            # Determine if it's a weekend or a weekday
                            if dates[x].weekday() >= 5:  # Weekend (Saturday=5, Sunday=6)
                                shift_type = list(shift_types_weekend.keys())[y]
                                shift_time = shift_types_weekend[shift_type]
                            else:  # Weekday
                                shift_type = list(shift_types.keys())[y]
                                shift_time = shift_types[shift_type]
                            self._results.append((dates[x], shift_type, shift_time, mitarbeitende_df['Mitarbeitenden_ID'].iloc[z]))
        self._solution_count += 1

    def get_dataframe(self):
        return pd.DataFrame(self._results, columns=['Datum', 'Schicht', 'Schichtzeit', 'Dienst'])


# Solve the model
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solutionrange = range(1)  # Display 1 feasible result (the first one)
solution_printer = SolutionPrinterClass(shiftoptions, workers, days, shifts, solutionrange)
solver.Solve(model, solution_printer)

# Get and display the results as a DataFrame
df = solution_printer.get_dataframe()

#show output in jupyter
xlsx = df.to_excel('Dienstplan.xlsx', index=False)
print(df.head(20))

: 

### Test

In [4]:
# Function to check if a worker works more than one shift per day
def check_multiple_shifts_per_day(df):
    for date in df['Datum'].unique():
        for worker in range(workers):
            shifts_worked = df[(df['Datum'] == date) & (df['Dienst'] == worker)].shape[0]
            if shifts_worked > 1:
                print(f"Worker {worker} worked {shifts_worked} shifts on {date}.")

# Call the function to check multiple shifts per day
check_multiple_shifts_per_day(df)

# Function to check for repeats in previous 5 rows of Dienst
def check_consecutive_repeats(df):
    for i in range(len(df)):
        current_dienst = df.iloc[i]['Dienst']
        previous_diensts = df.iloc[max(0, i-5):i]['Dienst'].tolist()
        if current_dienst in previous_diensts:
            print(f"Row {i}: Dienst value '{current_dienst}' has been repeated within the previous 5 rows.")

# Call the function to check repeats
check_consecutive_repeats(df)

# Function to check if 'WF' and 'U' constraints are respected
def test_wf_u_constraints(df, mitarbeitende_df):
    for index, row in mitarbeitende_df.iterrows():
        worker_id = row['Mitarbeitenden_ID']
        for date_column in mitarbeitende_df.columns[1:]:  # Skip the first column ('Mitarbeitenden_ID')
            if row[date_column] in ['WF', 'U']:
                # Check if any shift is assigned to this worker on the date
                if not df[(df['Dienst'] == worker_id) & (df['Datum'] == pd.to_datetime(date_column))].empty:
                    print(f"Error: Worker {worker_id} has a shift assigned on {date_column} despite being marked as '{row[date_column]}'.")

# Call the function to check Wunschfrei
test_wf_u_constraints(df, mitarbeitende_df)

NameError: name 'df' is not defined

In [21]:
#Change output dataframe
# Create a pivot table
pivot_df = df.pivot_table(index='Datum', columns='Dienst', values='Schicht', aggfunc='first')

# Fill NaN values with empty strings
pivot_df.fillna('', inplace=True)

# Flatten the column MultiIndex
pivot_df.columns = [str(col) for col in pivot_df.columns]

# Reset index to have 'Datum' as a column
pivot_df.reset_index(inplace=True)

# Reorder columns to start with 'Datum'
cols = ['Datum'] + [col for col in pivot_df.columns if col != 'Datum']
pivot_df = pivot_df[cols]

# Save the new DataFrame to Excel (optional)
pivot_df.to_excel('Dienstplan_pivot.xlsx', index=False)

# Display the new DataFrame
print(pivot_df.head(20))


        Datum     101 102     103     104     105      106      107     108  \
0  2024-07-01                                               Nacht 1           
1  2024-07-02                      Früh 2           Spät 2                    
2  2024-07-03                              Spät 1  Nacht 1                    
3  2024-07-04  Früh 2      Früh 1                                             
4  2024-07-05                                                                 
5  2024-07-06                                                                 
6  2024-07-07                                                                 
7  2024-07-08                                                                 
8  2024-07-09                                                                 
9  2024-07-10                                                                 
10 2024-07-11                                                                 
11 2024-07-12                                       

In [22]:
#Show workers hours overview
# Calculate shifts per worker per shift type
worker_shifts = df.groupby(['Dienst', 'Schicht'])['Datum'].count().unstack(fill_value=0)

# Calculate weekly hours per worker
hours_per_shift = {'Früh 1': 8, 'Früh 2': 8, 'Spät 1': 8, 'Spät 2': 8, 'Nacht 1': 8, 'Nacht 2': 8}  # Assuming each shift is 8 hours

# Aggregate shifts into single categories
shift_mapping = {
    'Früh': ['Früh 1', 'Früh 2'],
    'Spät': ['Spät 1', 'Spät 2'],
    'Nacht': ['Nacht 1', 'Nacht 2']
}

# Calculate weekly hours for each worker
worker_hours = {}
for worker in range(1, workers + 1):  # Assuming workers are 1-indexed
    weekly_hours = 0
    worker_stats = {}
    for shift_category, shifts in shift_mapping.items():
        total_shifts = sum(worker_shifts.get(shift, 0).get(worker, 0) for shift in shifts)
        total_hours = total_shifts * hours_per_shift[shifts[0]]  # All shifts are considered 8 hours
        weekly_hours += (total_hours / days) * 7  # Adjust 'days_in_month' accordingly
        worker_stats[shift_category] = total_shifts

    worker_stats['Wochenstunden'] = weekly_hours
    worker_hours[f'Mitarbeitende {worker}'] = worker_stats

# Convert worker_hours to DataFrame for a neat display
hours_df = pd.DataFrame(worker_hours).transpose()

# Format the DataFrame to display integers for Früh, Spät, Nacht and two decimal places for Wochenstunden
hours_df = hours_df.astype({'Früh': int, 'Spät': int, 'Nacht': int})
hours_df['Wochenstunden'] = hours_df['Wochenstunden'].round(2)

print(hours_df)

                  Früh  Spät  Nacht  Wochenstunden
Mitarbeitende 1      0     0      0            0.0
Mitarbeitende 2      0     0      0            0.0
Mitarbeitende 3      0     0      0            0.0
Mitarbeitende 4      0     0      0            0.0
Mitarbeitende 5      0     0      0            0.0
Mitarbeitende 6      0     0      0            0.0
Mitarbeitende 7      0     0      0            0.0
Mitarbeitende 8      0     0      0            0.0
Mitarbeitende 9      0     0      0            0.0
Mitarbeitende 10     0     0      0            0.0
Mitarbeitende 11     0     0      0            0.0
Mitarbeitende 12     0     0      0            0.0
Mitarbeitende 13     0     0      0            0.0
Mitarbeitende 14     0     0      0            0.0
Mitarbeitende 15     0     0      0            0.0
Mitarbeitende 16     0     0      0            0.0
Mitarbeitende 17     0     0      0            0.0
Mitarbeitende 18     0     0      0            0.0
Mitarbeitende 19     0     0   

### BACKUP

In [ ]:
#LAST WORKING CODE 2024-06-14 13:19

import pandas as pd
from ortools.sat.python import cp_model


# Month and year
month = 2
year = 2024

#Create the number of days per month
if month in [1, 3, 5, 7, 8, 10, 12]:
    num_days = 31
elif month in [4, 6, 9, 11]:
    num_days = 30
elif month == 2:
    num_days = 29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28
else:
    raise ValueError("Invalid month")

# Generate the date range
dates = pd.date_range(start=f'{year}-{month:02d}-01', periods=num_days)

# Declare parameters
workers = len(mitarbeitende_df['Mitarbeitenden_ID'])
shifts = 6
days = len(dates.strftime('%Y-%m-%d').tolist())
min_break = 6 # can't really be changed (i dont know why now)
maxshiftsperday = 1
maxdifference = 1 #difference between number of shifts per employee (needs to be changed later when hours are taken into consideration)

# Dictionary to map shift types to time ranges
shift_types = {
    'Früh 1': '06:00-14:00',
    'Früh 2': '06:00-14:00',
    'Spät 1': '14:00-22:00',
    'Spät 2': '14:00-22:00',
    'Nacht 1': '22:00-06:00',
    'Nacht 2': '22:00-06:00'
}

# Initialize model
model = cp_model.CpModel()

# Create shift options
shiftoptions = {}
for x in range(days):
    for y in range(shifts):
        for z in range(workers):
            shiftoptions[(x, y, z)] = model.NewBoolVar(f"shift_with_id_{x}_{y}_{z}")

# Constraint: Each shift is assigned to exactly one worker
for x in range(days):
    for y in range(shifts):
        model.Add(sum(shiftoptions[(x, y, z)] for z in range(workers)) == 1)

# Constraint: Each worker works at most one shift per day
for x in range(days):
    for z in range(workers):
        model.Add(sum(shiftoptions[(x, y, z)] for y in range(shifts)) <= maxshiftsperday)

# Add constraint ensuring at least a min_break of 6 shifts between consecutive shifts for each worker
# Add custom break constraints
for z in range(workers):
    for x in range(days):
        for y in range(shifts):
            for k in range(1, min_break):  # Adjust based on the break_time
                if y + k < shifts:
                    model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x, y + k, z)] <= 1)
                if x + 1 < days and y - k >= 0:
                    model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x + 1, y - k, z)] <= 1)

# Constraint: Balance the number of shifts per worker
minshiftsperworker = (shifts * days) // workers
maxshiftsperworker = minshiftsperworker + maxdifference
for z in range(workers):
    shiftsassigned = 0
    for x in range(days):
        for y in range(shifts):
            shiftsassigned += shiftoptions[(x, y, z)]
    model.Add(minshiftsperworker <= shiftsassigned)
    model.Add(shiftsassigned <= maxshiftsperworker)

# Custom solution printer to store results in a DataFrame
class SolutionPrinterClass(cp_model.CpSolverSolutionCallback):
    def __init__(self, shiftoptions, workers, days, shifts, sols):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shiftoptions = shiftoptions
        self._workers = workers
        self._days = days
        self._shifts = shifts
        self._solutions = set(sols)
        self._solution_count = 0
        self._results = []

    def on_solution_callback(self):
        if self._solution_count in self._solutions:
            for x in range(self._days):
                for y in range(self._shifts):
                    for z in range(self._workers):
                        if self.Value(self._shiftoptions[(x, y, z)]):
                            # Get the shift type and corresponding time range
                            shift_type = list(shift_types.keys())[y]
                            shift_time = shift_types[shift_type]
                            self._results.append((dates[x], shift_type, shift_time, mitarbeitende_df['Mitarbeitenden_ID'][z]))  # Add 1 to worker ID
        self._solution_count += 1

    def get_dataframe(self):
        return pd.DataFrame(self._results, columns=['Datum', 'Schicht', 'Schichtzeit', 'Dienst'])

# Solve the model
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solutionrange = range(1)  # Display 1 feasible result (the first one)
solution_printer = SolutionPrinterClass(shiftoptions, workers, days, shifts, solutionrange)
solver.Solve(model, solution_printer)

# Get and display the results as a DataFrame
df = solution_printer.get_dataframe()
xlsx = df.to_excel('Dienstplan.xlsx', index=False)
print(df.head(20))

In [ ]:
#BACKUP LAST WORKING CODE
import pandas as pd
from ortools.sat.python import cp_model

# Month and year
month = 2
year = 2024

#Create the number of days per month
if month in [1, 3, 5, 7, 8, 10, 12]:
    num_days = 31
elif month in [4, 6, 9, 11]:
    num_days = 30
elif month == 2:
    num_days = 29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28
else:
    raise ValueError("Invalid month")

# Generate the date range
dates = pd.date_range(start=f'{year}-{month:02d}-01', periods=num_days)

# Dictionary to map shift types to time ranges
shift_types = {
    'Früh 1': '06:00-14:00',
    'Früh 2': '06:00-14:00',
    'Spät 1': '14:00-22:00',
    'Spät 2': '14:00-22:00',
    'Nacht 1': '22:00-06:00',
    'Nacht 2': '22:00-06:00'
}

# Declare parameters
workers = 15
shifts = 6
days = len(dates.strftime('%Y-%m-%d').tolist())
min_break = 6
maxshiftsperday = 1
maxdifference = 1

# Initialize model
model = cp_model.CpModel()

# Create shift options
shiftoptions = {}
for x in range(days):
    for y in range(shifts):
        for z in range(workers):
            shiftoptions[(x, y, z)] = model.NewBoolVar(f"shift_with_id_{x}_{y}_{z}")

# Constraint: Each shift is assigned to exactly one worker
for x in range(days):
    for y in range(shifts):
        model.Add(sum(shiftoptions[(x, y, z)] for z in range(workers)) == 1)

# Constraint: Each worker works at most one shift per day
for x in range(days):
    for z in range(workers):
        model.Add(sum(shiftoptions[(x, y, z)] for y in range(shifts)) <= maxshiftsperday)

# Add constraint ensuring at least a min_break of 6 shifts between consecutive shifts for each worker
# Add custom break constraints
for z in range(workers):
    for x in range(days):
        for y in range(shifts):
            # y = 1
            if y == 0:
                for k in range(1, 6):
                    if y + k < shifts:
                        model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x, y + k, z)] <= 1)
            # y = 2
            elif y == 1:
                for k in range(1, 5):
                    if y + k < shifts:
                        model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x, y + k, z)] <= 1)
                if x + 1 < days:
                    model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x + 1, y - 1, z)] <= 1)
            # y = 3
            elif y == 2:
                for k in range(1, 4):
                    if y + k < shifts:
                        model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x, y + k, z)] <= 1)
                if x + 1 < days:
                    for k in range(1, 3):
                        model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x + 1, y - k, z)] <= 1)
            # y = 4
            elif y == 3:
                for k in range(1, 3):
                    if y + k < shifts:
                        model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x, y + k, z)] <= 1)
                if x + 1 < days:
                    for k in range(1, 4):
                        model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x + 1, y - k, z)] <= 1)
            # y = 5
            elif y == 4:
                if y + 1 < shifts:
                    model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x, y + 1, z)] <= 1)
                if x + 1 < days:
                    for k in range(1, 5):
                        model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x + 1, y - k, z)] <= 1)
            # y = 6
            elif y == 5:
                if x + 1 < days:
                    for k in range(1, 6):
                        model.Add(shiftoptions[(x, y, z)] + shiftoptions[(x + 1, y - k, z)] <= 1)

# Constraint: Balance the number of shifts per worker
minshiftsperworker = (shifts * days) // workers
maxshiftsperworker = minshiftsperworker + maxdifference
for z in range(workers):
    shiftsassigned = 0
    for x in range(days):
        for y in range(shifts):
            shiftsassigned += shiftoptions[(x, y, z)]
    model.Add(minshiftsperworker <= shiftsassigned)
    model.Add(shiftsassigned <= maxshiftsperworker)

# Custom solution printer to store results in a DataFrame
class SolutionPrinterClass(cp_model.CpSolverSolutionCallback):
    def __init__(self, shiftoptions, workers, days, shifts, sols):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shiftoptions = shiftoptions
        self._workers = workers
        self._days = days
        self._shifts = shifts
        self._solutions = set(sols)
        self._solution_count = 0
        self._results = []

    def on_solution_callback(self):
        if self._solution_count in self._solutions:
            for x in range(self._days):
                for y in range(self._shifts):
                    for z in range(self._workers):
                        if self.Value(self._shiftoptions[(x, y, z)]):
                            # Get the shift type and corresponding time range
                            shift_type = list(shift_types.keys())[y]
                            shift_time = shift_types[shift_type]
                            self._results.append((dates[x], shift_type, shift_time, z+1))  # Add 1 to worker ID
        self._solution_count += 1

    def get_dataframe(self):
        return pd.DataFrame(self._results, columns=['Datum', 'Schicht', 'Schichtzeit', 'Dienst'])

# Solve the model
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solutionrange = range(1)  # Display 1 feasible result (the first one)
solution_printer = SolutionPrinterClass(shiftoptions, workers, days, shifts, solutionrange)
solver.Solve(model, solution_printer)

# Get and display the results as a DataFrame
df = solution_printer.get_dataframe()
print(df.head(20))

In [ ]:
#CREATE TARGET DATAFRAME (not sure if needed anymore)

import pandas as pd
import numpy as np

# Load employee and working hours data
mitarbeitende_df = pd.read_csv('data/Input_Daten_Dienstplan - Mitarbeitende.csv')

# Example month and year
month = 2
year = 2024

if month in [1, 3, 5, 7, 8, 10, 12]:
    num_days = 31
elif month in [4, 6, 9, 11]:
    num_days = 30
elif month in [2]:
    num_days = 29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28
else:
    raise ValueError("Invalid month")

# Generate the date range
dates = pd.date_range(start=f'{year}-{month:02d}-01', periods=num_days)

# Define shifts
shift_types = {'Früh': '06:00-14:00', 'Spät': '14:00-22:00', 'Nacht': '22:00-06:00'}
# Legal break time between shifts
break_time = 2

# Generate the list of date and shift type combinations
dates_shifts = [(date, shift) for date in dates for shift in shift_types.keys()]

# Create Dienstplan DataFrame
Dienstplan_df = pd.DataFrame(dates_shifts, columns=['date', 'shift_type'])

# Map shift types to shift hours
Dienstplan_df['shift_hours'] = Dienstplan_df['shift_type'].map(shift_types)

# Add additional columns for shifts
Dienstplan_df['Dienst 1'] = ''
Dienstplan_df['Dienst 2'] = ''

print(Dienstplan_df.head(30))


### OUTTAKE

In [5]:
import pandas as pd
from ortools.sat.python import cp_model

# Define shifts and people
shifts = ['Morning', 'Afternoon', 'Night']
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
people = ['Alice', 'Bob', 'Charlie', 'David', 'Eva']

# Create a DataFrame to hold the schedule
schedule = pd.DataFrame(index=pd.MultiIndex.from_product([days, shifts], names=['Day', 'Shift']), columns=['Shift'])

# Define the availability (1 for available, 0 for not available)
availability = {
    'Alice': [1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1],
    'Bob': [1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
    'Charlie': [1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
    'David': [1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    'Eva': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
}

# Convert availability to DataFrame
availability_df = pd.DataFrame(availability, index=pd.MultiIndex.from_product([days, shifts], names=['Day', 'Shift']))

# Initialize the model
model = cp_model.CpModel()

# Define variables
shift_vars = {}
for day in days:
    for shift in shifts:
        shift_vars[(day, shift)] = [model.NewBoolVar(f'shift_{day}_{shift}_{person}') for person in people]

# Constraints
# Each shift is assigned to exactly one person
for day in days:
    for shift in shifts:
        model.AddExactlyOne(shift_vars[(day, shift)])

# Each person works at most one shift per day
for person in people:
    for day in days:
        person_shifts = [shift_vars[(day, shift)][people.index(person)] for shift in shifts]
        model.AddAtMostOne(person_shifts)

# Respect availability
for day in days:
    for shift in shifts:
        for person in people:
            if availability_df.loc[(day, shift), person] == 0:
                model.Add(shift_vars[(day, shift)][people.index(person)] == 0)

# Objective: Balance the number of shifts each person works
total_shifts = len(days) * len(shifts)
min_shifts = total_shifts // len(people)
max_shifts = (total_shifts + len(people) - 1) // len(people)
for person in people:
    num_shifts = sum(shift_vars[(day, shift)][people.index(person)] for day in days for shift in shifts)
    model.Add(min_shifts <= num_shifts)
    model.Add(num_shifts <= max_shifts)

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Output the schedule
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for day in days:
        for shift in shifts:
            assigned_person = None
            for person in people:
                if solver.Value(shift_vars[(day, shift)][people.index(person)]) == 1:
                    assigned_person = person
                    break
            schedule.loc[(day, shift), 'Shift'] = assigned_person
else:
    print('No solution found.')

print(schedule)

                       Shift
Day       Shift             
Monday    Morning    Charlie
          Afternoon      Eva
          Night        David
Tuesday   Morning      David
          Afternoon  Charlie
          Night          Eva
Wednesday Morning        Eva
          Afternoon      Bob
          Night        Alice
Thursday  Morning      David
          Afternoon      Bob
          Night        Alice
Friday    Morning    Charlie
          Afternoon      Bob
          Night        Alice


In [23]:
import pandas as pd
from ortools.sat.python import cp_model

# Define shifts and people
shifts = ['Morning', 'Afternoon', 'Night']
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
people = ['Alice', 'Bob', 'Charlie', 'David', 'Eva']

# Create a DataFrame to hold the schedule
schedule = pd.DataFrame(index=pd.MultiIndex.from_product([days, shifts], names=['Day', 'Shift']), columns=['Shift 1', 'Shift 2'])

# Define the availability (1 for available, 0 for not available)
availability = {
  'Alice': [1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1],
  'Bob': [1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
  'Charlie': [1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
  'David': [1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
  'Eva': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
}

# Convert availability to DataFrame
availability_df = pd.DataFrame(availability, index=pd.MultiIndex.from_product([days, shifts], names=['Day', 'Shift']))

# Initialize the model
model = cp_model.CpModel()

# Define variables
shift_vars1 = {}
shift_vars2 = {}
for day in days:
  for shift in shifts:
    shift_vars1[(day, shift)] = [model.NewBoolVar(f'shift1_{day}_{shift}_{person}') for person in people]
    shift_vars2[(day, shift)] = [model.NewBoolVar(f'shift2_{day}_{shift}_{person}') for person in people]

# Constraints
# Each person can be assigned to at most one shift (combined) per day
for day in days:
  for person in people:
    person_shifts = [shift_vars1[(day, shift)][people.index(person)] + shift_vars2[(day, shift)][people.index(person)] for shift in shifts]
    model.AddAtMostOne(person_shifts)

# Exactly two people are assigned to each shift
for day in days:
  for shift in shifts:
    assigned_people = [shift_vars1[(day, shift)][i] + shift_vars2[(day, shift)][i] for i in range(len(people))]
    model.AddExactlyOne(assigned_people)

# Respect availability
for day in days:
  for shift in shifts:
    for person in people:
      if availability_df.loc[(day, shift), person] == 0:
        model.Add(shift_vars1[(day, shift)][people.index(person)] == 0)
        model.Add(shift_vars2[(day, shift)][people.index(person)] == 0)

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Output the schedule
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  for day in days:
    for shift in shifts:
      assigned_person1 = None
      assigned_person2 = None
      for person in people:
        if solver.Value(shift_vars1[(day, shift)][people.index(person)]) == 1:
          assigned_person1 = person
        if solver.Value(shift_vars2[(day, shift)][people.index(person)]) == 1 and assigned_person1 != person:
          assigned_person2 = person
      schedule.loc[(day, shift), 'Shift 1'] = assigned_person1
      schedule.


SyntaxError: invalid syntax (4006372769.py, line 72)

### Sarah's code

In [34]:
import pandas as pd
import numpy as np

# Load employee and working hours data
mitarbeitende_df = pd.read_csv('data/Input_Daten_Dienstplan - Mitarbeitende.csv')

# Example month and year
month = 2
year = 2024

if month in [1, 3, 5, 7, 8, 10, 12]:
    num_days = 31
elif month in [4, 6, 9, 11]:
    num_days = 30
elif month in [2]:
    num_days = 29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28
else:
    raise ValueError("Invalid month")

# Generate the date range
dates = pd.date_range(start=f'{year}-{month:02d}-01', periods=num_days)

# Define shifts
shift_types = {'early': 8, 'late': 8, 'night': 10}
# Legal break time between shifts
break_time = 2

# Generate the list of date and shift type combinations
dates_shifts = [(date, shift) for date in dates for shift in shift_types.keys()]

# Create Dienstplan DataFrame
Dienstplan_df = pd.DataFrame(dates_shifts, columns=['date', 'shift_type'])

# Map shift types to shift hours
Dienstplan_df['shift_hours'] = Dienstplan_df['shift_type'].map(shift_types)

# Add additional columns for shifts
Dienstplan_df['Dienst 1'] = ''
Dienstplan_df['Dienst 2'] = ''


print(Dienstplan_df.head(30))


         date shift_type  shift_hours Dienst 1 Dienst 2
0  2024-02-01      early            8                  
1  2024-02-01       late            8                  
2  2024-02-01      night           10                  
3  2024-02-02      early            8                  
4  2024-02-02       late            8                  
5  2024-02-02      night           10                  
6  2024-02-03      early            8                  
7  2024-02-03       late            8                  
8  2024-02-03      night           10                  
9  2024-02-04      early            8                  
10 2024-02-04       late            8                  
11 2024-02-04      night           10                  
12 2024-02-05      early            8                  
13 2024-02-05       late            8                  
14 2024-02-05      night           10                  
15 2024-02-06      early            8                  
16 2024-02-06       late            8           

In [35]:
import random

# Get a list of available employee IDs
employee_ids = list(range(101, 120))

# Function to check if an employee can be assigned a shift
def can_assign_shift(employee_id, date, last_assigned):
    if last_assigned.get(employee_id) is None:
        return True
    else:
        last_date = last_assigned[employee_id]
        days_since_last = (date - last_date).days
        return days_since_last >= break_time

# Initialize a dictionary to track last assigned dates for each employee
last_assigned = {}

for index, row in Dienstplan_df.iterrows():
    date = row['date']

    # Assign Dienst 1
    available_employees = [emp_id for emp_id in employee_ids if can_assign_shift(emp_id, date, last_assigned)]
    if available_employees:
        employee_id = random.choice(available_employees)
        Dienstplan_df.at[index, 'Dienst 1'] = employee_id
        last_assigned[employee_id] = date

    # Assign Dienst 2, ensuring it's not the same employee as Dienst 1
    available_employees = [emp_id for emp_id in employee_ids if can_assign_shift(emp_id, date, last_assigned) and emp_id != Dienstplan_df.at[index, 'Dienst 1']]
    if available_employees:
        employee_id = random.choice(available_employees)
        Dienstplan_df.at[index, 'Dienst 2'] = employee_id
        last_assigned[employee_id] = date

print(Dienstplan_df.head(30))

         date shift_type  shift_hours Dienst 1 Dienst 2
0  2024-02-01      early            8      108      101
1  2024-02-01       late            8      104      107
2  2024-02-01      night           10      112      118
3  2024-02-02      early            8      109      111
4  2024-02-02       late            8      110      113
5  2024-02-02      night           10      117      102
6  2024-02-03      early            8      119      106
7  2024-02-03       late            8      101      104
8  2024-02-03      night           10      107      105
9  2024-02-04      early            8      118      108
10 2024-02-04       late            8      103      111
11 2024-02-04      night           10      112      114
12 2024-02-05      early            8      110      109
13 2024-02-05       late            8      115      119
14 2024-02-05      night           10      116      113
15 2024-02-06      early            8      102      112
16 2024-02-06       late            8      103  

In [18]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model

# Load employee and working hours data
mitarbeitende_df = pd.read_csv('data/Input_Daten_Dienstplan - Mitarbeitende.csv')

# Example month and year
month = 2
year = 2024

if month in [1, 3, 5, 7, 8, 10, 12]:
    num_days = 31
elif month in [4, 6, 9, 11]:
    num_days = 30
elif month in [2]:
    num_days = 29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28
else:
    raise ValueError("Invalid month")

# Generate the date range
dates = pd.date_range(start=f'{year}-{month:02d}-01', periods=num_days)

# Define shifts
shift_types = {'early': 8, 'late': 8, 'night': 10}
# Legal break time between shifts
break_time = 11

# Generate the list of date and shift type combinations
dates_shifts = [(date, shift) for date in dates for shift in shift_types.keys()]

# Create Dienstplan DataFrame
Dienstplan_df = pd.DataFrame(dates_shifts, columns=['date', 'shift_type'])

# Map shift types to shift hours
Dienstplan_df['shift_hours'] = Dienstplan_df['shift_type'].map(shift_types)

# Add additional columns for shifts
Dienstplan_df['Dienst 1'] = ''
Dienstplan_df['Dienst 2'] = ''

# Initialize the model
model = cp_model.CpModel()

# Define variables
# Define variables
shift_vars = {}
for idx, row in Dienstplan_df.iterrows():
    date, shift_type = row['date'], row['shift_type']
    shift_vars[(date, shift_type, 'Dienst 1')] = model.NewIntVar(0, len(mitarbeitende_df['Mitarbeitenden_ID']) - 1, f'shift_{date}_{shift_type}_Dienst_1')
    shift_vars[(date, shift_type, 'Dienst 2')] = model.NewIntVar(0, len(mitarbeitende_df['Mitarbeitenden_ID']) - 1, f'shift_{date}_{shift_type}_Dienst_2')

# Constraints
# Each shift is assigned to at most two employees
for date, shift_type in dates_shifts:
    model.AddAllDifferent([shift_vars[(date, shift_type, 'Dienst 1')], shift_vars[(date, shift_type, 'Dienst 2')]])

# Each employee works at most one shift per day
for employee_id in mitarbeitende_df['Mitarbeitenden_ID']:
    for date in dates:
        employee_shifts = []
        for shift_type in shift_types.keys():
            for dienst in ['Dienst 1', 'Dienst 2']:
                employee_shift_var = model.NewBoolVar(f'employee_{employee_id}_shift_{date}_{shift_type}_{dienst}')
                employee_shift_expr = (shift_vars[(date, shift_type, dienst)] == employee_id)
                model.Add(employee_shift_var == employee_shift_expr)
                employee_shifts.append(employee_shift_var)
        model.AddAtMostOne(employee_shifts)

# Add constraints for shift breaks
for employee_id in mitarbeitende_df['Mitarbeitenden_ID']:
    for date in dates:
        prev_shift_var = None
        for shift_type in shift_types.keys():
            for dienst in ['Dienst 1', 'Dienst 2']:
                var = shift_vars[(date, shift_type, dienst)]
                employee_shift_var = model.NewBoolVar(f'employee_{employee_id}_shift_{date}_{shift_type}_{dienst}')
                model.Add(employee_shift_var == (var == employee_id))
                if prev_shift_var is not None:
                    break_var = model.NewBoolVar(f'break_var_{date}_{shift_type}_{dienst}')
                    model.Add(var - prev_shift_var >= 2).OnlyEnforceIf(break_var)
                    prev_shift_not_zero_var = model.NewBoolVar(f'prev_shift_not_zero_var_{date}_{shift_type}_{dienst}')
                    model.Add(prev_shift_not_zero_var == (prev_shift_var != 0)).OnlyEnforceIf(prev_shift_var is not None)
                    enforce_break_var = model.NewBoolVar(f'enforce_break_var_{date}_{shift_type}_{dienst}')
                    model.Add(enforce_break_var == (break_var == True)).OnlyEnforceIf(prev_shift_not_zero_var)
                    model.Add(1 - enforce_break_var).OnlyEnforceIf(1 - prev_shift_not_zero_var)
                prev_shift_var = var

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Output the schedule
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for idx, row in Dienstplan_df.iterrows():
        date, shift_type = row['date'], row['shift_type']
        Dienstplan_df.at[idx, 'Dienst 1'] = solver.Value(shift_vars[(date, shift_type, 'Dienst 1')])
        Dienstplan_df.at[idx, 'Dienst 2'] = solver.Value(shift_vars[(date, shift_type, 'Dienst 2')])
else:
    print('No solution found.')

print(Dienstplan_df)

NotImplementedError: Evaluating a LinearExpr instance as a Boolean is not implemented.

### Testing

In [9]:
def check_conditions(Dienstplan_df):
    # Initialize a flag to indicate if any violations are found
    violations_found = False

    # Check condition 1: No employee is assigned to both Dienst 1 and Dienst 2 in the same slot
    condition_1_violations = Dienstplan_df[Dienstplan_df['Dienst 1'] == Dienstplan_df['Dienst 2']]
    if not condition_1_violations.empty:
        violations_found = True
        print("Condition 1 Violation: Employee assigned to both Dienst 1 and Dienst 2 in the same slot.")
        print(condition_1_violations)

    # Check condition 2: No employee is assigned to a slot if they were assigned to any slot in the last two slots
    condition_2_violations = []
    for i in range(2, len(Dienstplan_df)):
        current_slot = Dienstplan_df.iloc[i]
        prev_slots = Dienstplan_df.iloc[i-2:i]
        if current_slot['Dienst 1'] in prev_slots['Dienst 1'].values or current_slot['Dienst 1'] in prev_slots['Dienst 2'].values:
            condition_2_violations.append((i, 'Dienst 1', current_slot['Dienst 1']))
        if current_slot['Dienst 2'] in prev_slots['Dienst 1'].values or current_slot['Dienst 2'] in prev_slots['Dienst 2'].values:
            condition_2_violations.append((i, 'Dienst 2', current_slot['Dienst 2']))

    if condition_2_violations:
        violations_found = True
        print("Condition 2 Violation: Employee assigned to a slot after being assigned in the last two slots.")
        for violation in condition_2_violations:
            print(f"Index {violation[0]}: Employee {violation[2]} assigned to {violation[1]}")

    if not violations_found:
        print("No violations found. The schedule meets all conditions.")

# Example usage
# Assuming Dienstplan_df is the DataFrame generated from your scheduling logic
check_conditions(Dienstplan_df)


Condition 1 Violation: Employee assigned to both Dienst 1 and Dienst 2 in the same slot.
         date shift_type  shift_hours Dienst 1 Dienst 2
0  2024-02-01      early            8                  
1  2024-02-01       late            8                  
2  2024-02-01      night           10                  
3  2024-02-02      early            8                  
4  2024-02-02       late            8                  
..        ...        ...          ...      ...      ...
82 2024-02-28       late            8                  
83 2024-02-28      night           10                  
84 2024-02-29      early            8                  
85 2024-02-29       late            8                  
86 2024-02-29      night           10                  

[87 rows x 5 columns]
Condition 2 Violation: Employee assigned to a slot after being assigned in the last two slots.
Index 2: Employee  assigned to Dienst 1
Index 2: Employee  assigned to Dienst 2
Index 3: Employee  assigned to Dienst 1
I

### Inspiration Wunschfrei / Available slots of worker

In [87]:
class Worker(object):
    def __init__ (self, name, shifts, slots):
        self.name  = name
        self.shifts = shifts
        self.slots = slots  # Slot is a list of available slots a TA can fill

    def get_name(self):
        return self.name

    def get_shifts(self):
        return self.shifts

    def get_slots(self):
        return self.slots

    def copy(self):
        return Worker(self.name, self.shifts, self.slots[:])

    def assign(self, slot):
        assert slot in self.slots
        self.slots.remove(slot)
        self.shifts -= 1

    def __str__ (self):
        return  self.name + ', max-shifts = ' + str(self.shifts) + ', slots = ' + '[' + ','.join(str(e) for e in self.slots) +']'

def merge(left, right):
    ''' Helper function for the merge_sort function that follows'''
    result = []
    i,j = 0,0

    while i < len(left) and j < len(right):
        # This is the criterion merge_sort uses to sort the list argument
        # in this case I'll be sorting according to the length of list of slots for each Worker
        if len(left[i].get_slots()) < len(right[j].get_slots()):
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    while i < len(left):
        result.append(left[i])
        i += 1
    while j < len(right):
        result.append(right[j])
        j += 1

    return result

def merge_sort(L):
    ''' This sorts any List according to the condition in the func above'''
    if len(L) < 2:
        return L[:]
    else:
        middle = len(L)//2
        left  = merge_sort(L[:middle])
        right = merge_sort(L[middle:])
        return merge(left, right)

def poss_assignments(team, slots):
    """
    creates a list of possible assignments for every available slot

    team - List of Workers
    slots - List of positive integer (slots)

    return - Dictionary with slots as keys and a list of Workers as values
    """
    poss_dict = {}
    poss_slots = slots[:]


    for i in poss_slots:
        val_list = []
        poss_dict[i] = val_list
        for t in team:
            if i in t.get_slots():
                poss_dict[i] += [t]  #replace with [t.get_name()]: Use this to see the names this func produces

    return poss_dict

def Schedule(team, slots):
    team = merge_sort(team)
    if slots == []:
        return {}
    elif team == [] and slots != []:
        return print('No Workers to assign')

    possible_assignments = poss_assignments(team,slots) # A dictionary of slots with list of workers open for that slot

    accu_slot = slots[:]

    i = 0

    for slot in accu_slot:
       if slot in possible_assignments.keys():
           while i < len(possible_assignments[slot]):
               worker = possible_assignments[slot][i]
               wcp = worker.copy()
               wcp.assign(slot)

               new_slots = [s for s in slots if s != slot]
               new_team = [w if w != worker else wcp for w in team]

               assignment = Schedule(new_team, new_slots)

               if assignment != 'Schedule unattainable.':
                   assignment[slot] = wcp.get_name()
                   return assignment
               else:
                   i += 1

       else:

           break

    return 'Schedule unattainable.'

# Udon = Worker('Udon', 1, [3,4])
# Ramen = Worker('Ramen', 1, [2])
# Soba = Worker('Soba' , 2, [1,3])

# Noodle_workers = [Soba, Ramen, Udon]
# Slots = [1, 2, 3, 4]

#==============================================================================
Erika  = Worker("Erika",   1, [1, 3, 7, 9])
Ryan   = Worker("Ryan",    1,   [1, 8, 10])
Reece  = Worker("Reece",   2,       [5, 6])
Gordon = Worker("Gordon",  2,    [2, 3, 9])
David  = Worker("David",   2,    [2, 8, 9])
Katie  = Worker("Katie",   1,       [4, 6])
Aashish= Worker("Aashish", 2,      [1, 10])
Grant  = Worker("Grant",   2,      [1, 11])
Raeanne= Worker("Raeanne", 2,  [1, 11, 12])
Erin   = Worker("Erin",    1,          [4])
Alex   = Worker("Alex",    1,          [7])

Workers = [Erika, David, Grant, Raeanne, Ryan, Reece, Gordon, Katie, Aashish]
Slots = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

print(Schedule(Workers, Slots))
# print(Schedule(Noodle_workers, Slots))

{12: 'Raeanne', 11: 'Grant', 10: 'Aashish', 9: 'David', 8: 'David', 7: 'Erika', 6: 'Reece', 5: 'Reece', 4: 'Katie', 3: 'Gordon', 2: 'David', 1: 'Aashish'}


In [30]:
from ortools.sat.python import cp_model

num_nurses = 4
num_shifts = 3
num_days = 3
all_nurses = range(num_nurses)
all_shifts = range(num_shifts)
all_days = range(num_days)

model = cp_model.CpModel()

shifts = {}
for n in all_nurses:
    for d in all_days:
        for s in all_shifts:
            shifts[(n,d,s)] = model.new_bool_var(f"shift_n{n}_d{d}_s{s}")

for d in all_days:
    for s in all_shifts:
        model.add_exactly_one(shifts[(n,d,s)] for n in all_nurses)

for n in all_nurses:
    for d in all_days:
        model.add_at_most_one(shifts[(n, d, s)] for s in all_shifts)

# Try to distribute the shifts evenly, so that each nurse works
# min_shifts_per_nurse shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of nurses, some nurses will
# be assigned one more shift.

min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
if num_shifts * num_days % num_nurses == 0:
    max_shifts_per_nurse = min_shifts_per_nurse
else:
    max_shifts_per_nurse = min_shifts_per_nurse + 1
for n in all_nurses:
    shifts_worked =[]
    for d in all_days:
        for s in all_shifts:
            shifts_worked.append(shifts[(n,d,s)])
        model.add(min_shifts_per_nurse <= sum(shifts_worked))
        model.add(sum(shifts_worked) <= max_shifts_per_nurse)

solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True


class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_nurses = num_nurses
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        for d in range(self._num_days):
            print(f"Day {d}")
            for n in range(self._num_nurses):
                is_working = False
                for s in range(self._num_shifts):
                    if self.value(self._shifts[(n, d, s)]):
                        is_working = True
                        print(f"  Nurse {n} works shift {s}")
                if not is_working:
                    print(f"  Nurse {n} does not work")
        if self._solution_count >= self._solution_limit:
            print(f"Stop search after {self._solution_limit} solutions")
            self.stop_search()

    def solutionCount(self):
        return self._solution_count

# Display the first five solutions.
solution_limit = 5
solution_printer = NursesPartialSolutionPrinter(
    shifts, num_nurses, num_days, num_shifts, solution_limit
)
solver.solve(model, solution_printer)


3

In [31]:
"""Example of a simple nurse scheduling problem."""
from ortools.sat.python import cp_model


def main() -> None:
    # Data.
    num_nurses = 4
    num_shifts = 3
    num_days = 3
    all_nurses = range(num_nurses)
    all_shifts = range(num_shifts)
    all_days = range(num_days)

    # Creates the model.
    model = cp_model.CpModel()

    # Creates shift variables.
    # shifts[(n, d, s)]: nurse 'n' works shift 's' on day 'd'.
    shifts = {}
    for n in all_nurses:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d, s)] = model.new_bool_var(f"shift_n{n}_d{d}_s{s}")

    # Each shift is assigned to exactly one nurse in the schedule period.
    for d in all_days:
        for s in all_shifts:
            model.add_exactly_one(shifts[(n, d, s)] for n in all_nurses)

    # Each nurse works at most one shift per day.
    for n in all_nurses:
        for d in all_days:
            model.add_at_most_one(shifts[(n, d, s)] for s in all_shifts)

    # Try to distribute the shifts evenly, so that each nurse works
    # min_shifts_per_nurse shifts. If this is not possible, because the total
    # number of shifts is not divisible by the number of nurses, some nurses will
    # be assigned one more shift.
    min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
    if num_shifts * num_days % num_nurses == 0:
        max_shifts_per_nurse = min_shifts_per_nurse
    else:
        max_shifts_per_nurse = min_shifts_per_nurse + 1
    for n in all_nurses:
        shifts_worked = []
        for d in all_days:
            for s in all_shifts:
                shifts_worked.append(shifts[(n, d, s)])
        model.add(min_shifts_per_nurse <= sum(shifts_worked))
        model.add(sum(shifts_worked) <= max_shifts_per_nurse)

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.linearization_level = 0
    # Enumerate all solutions.
    solver.parameters.enumerate_all_solutions = True

    class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
        """Print intermediate solutions."""

        def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self._shifts = shifts
            self._num_nurses = num_nurses
            self._num_days = num_days
            self._num_shifts = num_shifts
            self._solution_count = 0
            self._solution_limit = limit

        def on_solution_callback(self):
            self._solution_count += 1
            print(f"Solution {self._solution_count}")
            for d in range(self._num_days):
                print(f"Day {d}")
                for n in range(self._num_nurses):
                    is_working = False
                    for s in range(self._num_shifts):
                        if self.value(self._shifts[(n, d, s)]):
                            is_working = True
                            print(f"  Nurse {n} works shift {s}")
                    if not is_working:
                        print(f"  Nurse {n} does not work")
            if self._solution_count >= self._solution_limit:
                print(f"Stop search after {self._solution_limit} solutions")
                self.stop_search()

        def solutionCount(self):
            return self._solution_count

    # Display the first five solutions.
    solution_limit = 5
    solution_printer = NursesPartialSolutionPrinter(
        shifts, num_nurses, num_days, num_shifts, solution_limit
    )

    solver.solve(model, solution_printer)

    # Statistics.
    print("\nStatistics")
    print(f"  - conflicts      : {solver.num_conflicts}")
    print(f"  - branches       : {solver.num_branches}")
    print(f"  - wall time      : {solver.wall_time} s")
    print(f"  - solutions found: {solution_printer.solutionCount()}")


if __name__ == "__main__":
    main()

Solution 1
Day 0
  Nurse 0 does not work
  Nurse 1 works shift 0
  Nurse 2 works shift 1
  Nurse 3 works shift 2
Day 1
  Nurse 0 works shift 2
  Nurse 1 does not work
  Nurse 2 works shift 1
  Nurse 3 works shift 0
Day 2
  Nurse 0 works shift 2
  Nurse 1 works shift 1
  Nurse 2 works shift 0
  Nurse 3 does not work
Solution 2
Day 0
  Nurse 0 works shift 0
  Nurse 1 does not work
  Nurse 2 works shift 1
  Nurse 3 works shift 2
Day 1
  Nurse 0 does not work
  Nurse 1 works shift 2
  Nurse 2 works shift 1
  Nurse 3 works shift 0
Day 2
  Nurse 0 works shift 2
  Nurse 1 works shift 1
  Nurse 2 works shift 0
  Nurse 3 does not work
Solution 3
Day 0
  Nurse 0 works shift 0
  Nurse 1 does not work
  Nurse 2 works shift 1
  Nurse 3 works shift 2
Day 1
  Nurse 0 works shift 1
  Nurse 1 works shift 2
  Nurse 2 does not work
  Nurse 3 works shift 0
Day 2
  Nurse 0 works shift 2
  Nurse 1 works shift 1
  Nurse 2 works shift 0
  Nurse 3 does not work
Solution 4
Day 0
  Nurse 0 works shift 0
  Nurse 